
# I - Configuração do ambiente


## 1.1 - Montagem do drive

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


## 1.2 - Escolha do ambiente e carregamento
São disponibilizadas duas opções de execução (dependendo da credencial utilizada)
 - Leitura: disponível para analises e usuários que não possuem permissão de escrita;
 - Escrita: disponível para atualização de bases (testes de desenvolvimento ou ambiente de produção).

In [ ]:
import os
if os.path.isfile('/content/monitor-rosa-leitura.json'):
    datalake_mode = 'leitura'
    %env SERVICE_ACCOUNT_USER=acesso-leitura@monitor-rosa.iam.gserviceaccount.com
    %env SERVICE_ACCOUNT_JSON=/content/monitor-rosa-leitura.json
elif os.path.isfile('/content/monitor-rosa-escrita.json'):
    datalake_mode = 'escrita'
    %env SERVICE_ACCOUNT_USER=acesso-escrita@monitor-rosa.iam.gserviceaccount.com
    %env SERVICE_ACCOUNT_JSON=/content/monitor-rosa-escrita.json
else:
    assert(os.path.isdir('/content/drive/Shareddrives/monitor-rosa-gold') == True)
    datalake_mode = 'shared_drive'
    %env SERVICE_ACCOUNT_USER=''
    %env SERVICE_ACCOUNT_JSON=''
datalake_mode

env: SERVICE_ACCOUNT_USER=''
env: SERVICE_ACCOUNT_JSON=''


'shared_drive'

In [ ]:
!rm -r sus-kpis-analysis
!git clone https://github.com/heber-augusto/sus-kpis-analysis.git

rm: cannot remove 'sus-kpis-analysis': No such file or directory
Cloning into 'sus-kpis-analysis'...
remote: Enumerating objects: 1779, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 1779 (delta 20), reused 12 (delta 12), pack-reused 1740 (from 3)
Receiving objects: 100% (1779/1779), 4.80 MiB | 11.71 MiB/s, done.
Resolving deltas: 100% (820/820), done.


## 1.3 - Instalação de libs Python, inicialização de variáveis de ambiente e configuração/instalação do Spark

In [ ]:
!pip install -r /content/sus-kpis-analysis/sia/etls/requirements.txt

%env PYTHONHASHSEED=1234
%env JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
%env SPARK_HOME=/content/spark-3.4.4-bin-hadoop3
%env SPARK_VERSION=3.4.4

!source /content/sus-kpis-analysis/sia/etls/bin/setup_spark_env.sh '/content/'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 67.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.4-py2.py3-none-any.whl size=311905460 sha256=5594db732f5ccd89184d955735a278c2380ba64070eeff95ce9143ceee97e2df
  Stored in directory: /root/.cache/pip/wheels/6b/0a/a1/2b8f5f192c7df9fdceb8e5a62873d64e46b101f980519bcf55
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.4
    Uninstalling pyspark-3.5.4:
      Successfully uninstalled pyspark-3.5.4
env: PYTHONHASHSEED=1234
env: JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
env: SPARK_HOME=/content/spark-3.4.4-bin-hadoop3
env: SPARK_VERSION=3.4.4
starting spark env setup 
installing and downloading packages
W: Skipping acquire of configured file 'main/source/Sources

In [ ]:
if datalake_mode != 'shared_drive':
    %env XDG_CONFIG_HOME=/content/datalake
    !source /content/sus-kpis-analysis/sia/etls/bin/install-google-drive-ocamlfuse.sh
    !source /content/sus-kpis-analysis/sia/etls/bin/mount_google_drive_v2.sh '/content/datalake' $SERVICE_ACCOUNT_USER '0ABIY-a4qrdY9Uk9PVA' 'monitor-rosa-bronze' $SERVICE_ACCOUNT_JSON '/content'
    !source /content/sus-kpis-analysis/sia/etls/bin/mount_google_drive_v2.sh '/content/datalake' $SERVICE_ACCOUNT_USER '0ALl0owLNr53oUk9PVA' 'monitor-rosa-silver' $SERVICE_ACCOUNT_JSON '/content'
    !source /content/sus-kpis-analysis/sia/etls/bin/mount_google_drive_v2.sh '/content/datalake' $SERVICE_ACCOUNT_USER '0AMHp9pBeLvZiUk9PVA' 'monitor-rosa-gold' $SERVICE_ACCOUNT_JSON '/content'

## 1.4 - Inicializa variáveis de acesso ao delta lake criado no drive




> O caminho do warehouse pode ser alterado em caso de testes de escritas locais.


In [ ]:
import os

lake_prefix = "temp-output"

if datalake_mode in ('leitura','shared_drive',):
    warehouse_dir = f"/content/datalake/{lake_prefix}/"

if datalake_mode == 'escrita':
    warehouse_dir = f"/content/datalake/"

spark_path = os.getenv('SPARK_HOME')
spark_path

'/content/spark-3.4.4-bin-hadoop3'

## 1.5 - Inclusão da pasta do repositório no python path

Procedimento permite que funções e classes presentes no repositório sejam utilizadas

In [ ]:
import sys
sys.path.append('/content/sus-kpis-analysis')
sys.path

['/content',
 '/env/python',
 '/usr/lib/python311.zip',
 '/usr/lib/python3.11',
 '/usr/lib/python3.11/lib-dynload',
 '',
 '/usr/local/lib/python3.11/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.11/dist-packages/IPython/extensions',
 '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor',
 '/root/.ipython',
 '/content/sus-kpis-analysis']

## 1.6 - Importação de funções utilizadas pelo código

In [ ]:
from sia.etls.lib.catalog_loader import DeltaLakeDatabaseFsCreator, load_entire_catalog_fs_v2
from sia.etls.lib.table_utilities import vacuum_tables_from_database, table_exists
from sia.etls.lib.fs_spark_session import create_fs_spark_session
from sia.etls.lib.bronze_files_utilities import get_pending_files_from_bronze
from sia.etls.lib.delta_table_creators import ParquetToDelta

## 1.7 - Cria Sessão Spark conectada ao Delta Lake presente no Drive

In [ ]:
spark = create_fs_spark_session(
    warehouse_dir=warehouse_dir,
    spark_path=spark_path
)

## 1.8 - Refresh do catálogo para utilizar consultas

In [ ]:
zone_names = ['monitor-rosa-bronze','monitor-rosa-silver','monitor-rosa-gold']

if datalake_mode in ('leitura', 'escrita'):

    zone_paths = [f'/content/datalake/{zone_name}/databases' for zone_name in zone_names]
else:
    zone_paths = [f'/content/drive/Shareddrives/{zone_name}/databases' for zone_name in zone_names]



# Carrega catalogo de banco de dados, na zona bronze
database_filter = None #['cnes_bronze.db',]

table_filter = None #['sia_bronze.ar','sia_bronze.aq', 'ibge_silver.cadastro_municipios', 'ibge_silver.demografia_municipios' ]

for databases_path in zone_paths:
    load_entire_catalog_fs_v2(
        spark_session = spark,
        databases_path = databases_path,
        use_db_folder_path=(datalake_mode == 'escrita'),
        database_filter=database_filter,
        table_filter=table_filter
    )

['sia_bronze.db', 'cnes_bronze.db', 'sih_bronze.db', 'sim_bronze.db']
Banco de dados sia_bronze criado.
listando conteúdos do caminho /content/drive/Shareddrives/monitor-rosa-bronze/databases e database sia_bronze
prefix: /content/drive/Shareddrives/monitor-rosa-bronze/databases/sia_bronze.db/
table_list: ['ar', 'aq', 'pa', 'bi', 'am']
Tabela ar criada
Tabela ar criada com comando CREATE TABLE IF NOT EXISTS sia_bronze.ar USING delta LOCATION '/content/drive/Shareddrives/monitor-rosa-bronze/databases/sia_bronze.db/ar'
Tabela aq criada
Tabela aq criada com comando CREATE TABLE IF NOT EXISTS sia_bronze.aq USING delta LOCATION '/content/drive/Shareddrives/monitor-rosa-bronze/databases/sia_bronze.db/aq'
Tabela pa criada
Tabela pa criada com comando CREATE TABLE IF NOT EXISTS sia_bronze.pa USING delta LOCATION '/content/drive/Shareddrives/monitor-rosa-bronze/databases/sia_bronze.db/pa'
Tabela bi criada
Tabela bi criada com comando CREATE TABLE IF NOT EXISTS sia_bronze.bi USING delta LOCATION

## 1.9 - Cria banco de dados gold


In [ ]:
destination_database_name = 'cancer_mama_1'

warehouse_dir_g =  f"/content/datalake/{lake_prefix}/"


db_creator = DeltaLakeDatabaseFsCreator(
    spark_session= spark,
    database_location=warehouse_dir_g,
    database_name=destination_database_name
)
db_creator.create_database(
     use_db_folder_path=(datalake_mode == 'escrita')
)

Banco de dados cancer_mama_1 criado.


# II - Exemplo de como listar bancos e tabelas

In [ ]:
databases = spark.sql(f"SHOW DATABASES;")
databases.show()

+-------------+
|    namespace|
+-------------+
|  cancer_data|
|cancer_mama_1|
|  cnes_bronze|
|      default|
|  ibge_silver|
|   sia_bronze|
|   sih_bronze|
|   sim_bronze|
+-------------+



In [ ]:
for row in databases.collect():
    spark.sql(f"SHOW TABLES FROM {row['namespace']};").show(truncate=False)

+-----------+-------------------------+-----------+
|namespace  |tableName                |isTemporary|
+-----------+-------------------------+-----------+
|cancer_data|aq_filtered              |false      |
|cancer_data|ar_filtered              |false      |
|cancer_data|cadastro_municipios      |false      |
|cancer_data|dados_estados_mensal     |false      |
|cancer_data|dados_municipios_mensal  |false      |
|cancer_data|demografia_municipios    |false      |
|cancer_data|pacientes                |false      |
|cancer_data|procedimentos            |false      |
|cancer_data|procedimentos_e_pacientes|false      |
+-----------+-------------------------+-----------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+

+-----------+---------+-----------+
|namespace  |tableName|isTemporary|
+-----------+---------+-----------+
|cnes_bronze|dc       |false      |
|cnes_bronze|ep       |false      |
|cnes_

# III - Extração e Filtragem:



## 3.1 - Definindo utils, variáveis e filtros

In [ ]:
def get_select_all_query(table_name, where_clause=''):
    return f"""
    SELECT
        *
    FROM {table_name}
    {where_clause}
    """

def run_sql_query(sql_query):
    return spark.sql(sql_query)

In [ ]:
# Definindo variavel destination_database_name

destination_database_name = 'cancer_mama_1'

# Filtro pelo CID

cid_filter = ['C500', 'C501', 'C502', 'C503', 'C504', 'C505', 'C506', 'C508', 'C509']

cid_filter = f"""({','.join([f"'{cid_id}'" for cid_id in cid_filter])})"""

proc_id_dict = {
    '0201010569': 'BIOPSIA/EXERESE DE NÓDULO DE MAMA',
    '0201010585': 'PUNÇÃO ASPIRATIVA DE MAMA POR AGULHA FINA',
    '0201010607': 'PUNÇÃO DE MAMA POR AGULHA GROSSA',
    '0203010035': 'EXAME DE CITOLOGIA (EXCETO CERVICO-VAGINAL E DE MAMA)',
    '0203010043': 'EXAME CITOPATOLOGICO DE MAMA',
    '0203020065': 'EXAME ANATOMOPATOLOGICO DE MAMA - BIOPSIA',
    '0203020073': 'EXAME ANATOMOPATOLOGICO DE MAMA - PECA CIRURGICA',
    '0205020097': 'ULTRASSONOGRAFIA MAMARIA BILATERAL',
    '0208090037': 'CINTILOGRAFIA DE MAMA (BILATERAL)',
    '0204030030': 'MAMOGRAFIA',
    '0204030188': 'MAMOGRAFIA BILATERAL PARA RASTREAMENTO'
}

proc_id_filter = f"""({','.join([f"'{proc_id}'" for proc_id in proc_id_dict.keys()])})"""

print(proc_id_filter)
print(cid_filter)

('0201010569','0201010585','0201010607','0203010035','0203010043','0203020065','0203020073','0205020097','0208090037','0204030030','0204030188')
('C500','C501','C502','C503','C504','C505','C506','C508','C509')


## 3.2 - Carrega Tabela de Cadastro de Municípios

In [ ]:
query_cadastro_municipios = spark.sql("""

SELECT *
FROM
ibge_silver.cadastro_municipios

""")

## 3.2 - Carrega tabela SIA.AR filtrando dados de câncer de mama e procedimentos de interesse

In [ ]:
sql_query_ar = get_select_all_query(
    table_name='sia_bronze.ar',
    where_clause=f"""
        WHERE AP_CIDPRI IN {cid_filter}
        """
)

cancer_ar_filtered = run_sql_query(sql_query_ar)

In [ ]:
cancer_ar_filtered\
      .repartition(1)\
      .write\
      .format("delta")\
      .mode("overwrite")\
      .saveAsTable(f"{destination_database_name}.ar_filtered")

## 3.3 - Carrega tabela SIA.AQ filtrando dados de câncer de mama e procedimentos de interesse

In [ ]:
sql_query_aq = get_select_all_query(
    table_name='sia_bronze.aq',
    where_clause=f"""
        WHERE AP_CIDPRI IN {cid_filter}
    """
)

cancer_aq_filtered = run_sql_query(sql_query_aq)

In [ ]:
cancer_aq_filtered\
      .repartition(1)\
      .write\
      .format("delta")\
      .mode("overwrite")\
      .saveAsTable(f"{destination_database_name}.aq_filtered")

# IV - Processamento dos Dados dos Pacientes e Procedimentos

## 4.1 - Cria dados consolidados de pacientes e procedimentos (quimio e radioterapia)

Radioterapia

In [ ]:
cancer_ar_res = spark.sql(f"""
SELECT
    AP_CMP as data,
    AP_CNSPCN as paciente,
    AR_ESTADI as estadiamento,
    DOUBLE(AP_VL_AP)  as custo,
    INT(AP_OBITO) as obito,
    AP_MUNPCN as municipio
FROM {destination_database_name}.ar_filtered
""")
cancer_ar_res.show(3)

+------+---------------+------------+------+-----+---------+
|  data|       paciente|estadiamento| custo|obito|municipio|
+------+---------------+------------+------+-----+---------+
|202404|{|{~}{~~|           0|5904.0|    0|   354100|
|202405|{{{~|~||           4|5904.0|    0|   351620|
|202404|{|{~}|           3|5904.0|    0|   355030|
+------+---------------+------------+------+-----+---------+
only showing top 3 rows



Quimio

In [ ]:
cancer_aq_res = spark.sql(f"""
SELECT
    AP_CMP as data,
    AP_CNSPCN as paciente,
    AQ_ESTADI as estadiamento,
    DOUBLE(AP_VL_AP)  as custo,
    INT(AP_OBITO) as obito,
    AP_MUNPCN as municipio
FROM {destination_database_name}.aq_filtered
""")
cancer_aq_res.show(3)

+------+---------------+------------+------+-----+---------+
|  data|       paciente|estadiamento| custo|obito|municipio|
+------+---------------+------------+------+-----+---------+
|202405|{{{{}|{}|           3| 79.75|    0|   351750|
|202405|{}|{||           3|1400.0|    0|   355030|
|202405|{|{}|{|           1| 79.75|    0|   355030|
+------+---------------+------------+------+-----+---------+
only showing top 3 rows



## 4.2 - Unifica os dados de radio e quimio consolidados

In [ ]:
df_union = cancer_aq_res.union(cancer_ar_res)

df_union.createOrReplaceTempView("cancer_ordered")

In [ ]:
df_union\
  .repartition(1)\
  .write\
  .format("delta")\
  .mode("overwrite")\
  .saveAsTable(f"{destination_database_name}.procedimentos")

## 4.3 - Consolidando os dados por paciente

In [ ]:
res_consolidado = spark.sql("""
SELECT
    paciente,
    FIRST(data) as data_primeiro_estadiamento,
    LAST(data) as data_ultimo_estadiamento,
    COUNT(1) as numero_procedimentos,
    FIRST(estadiamento) as primeiro_estadiamento,
    LAST(estadiamento) as ultimo_estadiamento,
    MAX (estadiamento) as maior_estadiamento,
    MIN (estadiamento) as menor_estadiamento,
    SUM(custo) as custo_total,
    MAX(obito) as indicacao_obito,
    FIRST(municipio) as primeiro_municipio,
    LAST(municipio) as ultimo_municipio
FROM (SELECT * FROM cancer_ordered ORDER BY paciente, data)
GROUP BY paciente
""")

res_consolidado\
  .repartition(1)\
  .write\
  .format("delta")\
  .mode("overwrite")\
  .saveAsTable(f"{destination_database_name}.pacientes")

## 4.4 - Procedimentos e Pacientes

In [ ]:
procedimentos_e_pacientes = spark.sql(f"""
  SELECT
      c.*,
      p.data_primeiro_estadiamento,
      p.data_ultimo_estadiamento,
      p.primeiro_estadiamento,
      p.maior_estadiamento,
      p.ultimo_estadiamento,
      p.custo_total,
      p.primeiro_municipio,
      p.ultimo_municipio,
      p.indicacao_obito
  FROM {destination_database_name}.procedimentos AS c
  FULL OUTER JOIN {destination_database_name}.pacientes AS p
  ON c.paciente = p.paciente
""")

procedimentos_e_pacientes\
  .repartition(1)\
  .write\
  .format("delta")\
  .mode("overwrite")\
  .saveAsTable(f"{destination_database_name}.procedimentos_e_pacientes")

In [ ]:
procedimentos_e_pacientes.show(3)

+------+--------+------------+-----+-----+---------+--------------------------+------------------------+---------------------+------------------+-------------------+-----------+------------------+----------------+---------------+
|  data|paciente|estadiamento|custo|obito|municipio|data_primeiro_estadiamento|data_ultimo_estadiamento|primeiro_estadiamento|maior_estadiamento|ultimo_estadiamento|custo_total|primeiro_municipio|ultimo_municipio|indicacao_obito|
+------+--------+------------+-----+-----+---------+--------------------------+------------------------+---------------------+------------------+-------------------+-----------+------------------+----------------+---------------+
|202405|        |           1|79.75|    0|   354890|                    202405|                  202405|                    1|                 4|                  3|   75437.05|            354890|          220770|              0|
|202405|        |           2|79.75|    0|   354890|                    202405| 

# V - Agregação por Município e Estado

## 5.1 - Consolida dados por municipio


In [ ]:
diagnosticos_por_estadiamento_municipio_df = spark.sql(f"""
    SELECT
        primeiro_estadiamento,
        data_primeiro_estadiamento AS data,
        primeiro_municipio AS municipio,
        COUNT(DISTINCT(paciente)) AS numero_diagnosticos
    FROM {destination_database_name}.pacientes
    WHERE primeiro_estadiamento != ''
    GROUP BY primeiro_estadiamento, data_primeiro_estadiamento, primeiro_municipio
""")



diagnosticos_por_estadiamento_municipio_df.createOrReplaceTempView("diagnosticos_por_estadiamento_municipio")

## 5.2 - Consolida dados mensais por municipio

In [ ]:
dados_estad_municipio_mensal_df = spark.sql(f"""
    SELECT
        data,
        municipio,
        primeiro_estadiamento,
        SUM(custo) AS custo_estadiamento,
        COUNT(DISTINCT(paciente)) AS numero_pacientes,
        SUM(DISTINCT(obito)) AS obitos,
        SUM(DISTINCT(indicacao_obito)) AS obito_futuro,
        COUNT(1) AS numero_procedimentos
    FROM
        (SELECT * FROM {destination_database_name}.procedimentos_e_pacientes ORDER BY data)
    GROUP BY data, municipio, primeiro_estadiamento
""")



dados_estad_municipio_mensal_df.createOrReplaceTempView("dados_municipios_mensal")

## 5.3 - Consolida dados por municipio mensal

In [ ]:
dados_estad_municipio_mensal = spark.sql("""
    SELECT
        mm.*,
        COALESCE(em.numero_diagnosticos, 0) AS numero_diagnosticos
    FROM dados_municipios_mensal mm
    FULL OUTER JOIN diagnosticos_por_estadiamento_municipio em
    ON mm.data = em.data
    AND mm.municipio = em.municipio
    AND mm.primeiro_estadiamento = em.primeiro_estadiamento
""")



dados_estad_municipio_mensal\
    .repartition(1)\
    .write\
    .format("delta")\
    .mode("overwrite")\
    .saveAsTable(f"{destination_database_name}.dados_estad_municipio_mensal")

## 5.4 - Agregação por estado

In [ ]:
dados_estad_mensal = spark.sql(f"""
    SELECT
        estado,
        data,
        primeiro_estadiamento,
        SUM(custo_estadiamento) AS custo_estadiamento,
        SUM(numero_pacientes) AS numero_pacientes,
        COUNT(DISTINCT(municipio)) AS numero_municipios,
        SUM(obitos) AS obitos,
        SUM(obito_futuro) AS obitos_futuros,
        SUM(numero_procedimentos) AS numero_procedimentos,
        SUM(numero_diagnosticos) AS numero_diagnosticos
    FROM (
        SELECT
            cadastro_cidades.nome_uf AS estado,
            mm.*
        FROM cancer_data.dados_municipios_mensal mm
        LEFT JOIN cancer_data.cadastro_municipios AS cadastro_cidades
        ON int(mm.municipio) = int(cadastro_cidades.id / 10)
        ORDER BY data
    ) AS dados_estado
    GROUP BY estado, data, primeiro_estadiamento
""")

dados_estad_mensal\
    .repartition(1)\
    .write\
    .format("delta")\
    .mode("overwrite")\
    .saveAsTable(f"{destination_database_name}.dados_estados_mensal")

# VI - Limpeza das tabelas delta, considerando 24 horas de retenção


In [ ]:
vacuum_tables_from_database(
        spark_session = spark,
        database_name = destination_database_name,
        retention_hours = 24
    )

vacuum completed for table cancer_mama_1.aq_filtered
vacuum completed for table cancer_mama_1.ar_filtered
vacuum completed for table cancer_mama_1.dados_estad_municipio_mensal
vacuum completed for table cancer_mama_1.dados_estados_mensal
vacuum completed for table cancer_mama_1.pacientes
vacuum completed for table cancer_mama_1.procedimentos
vacuum completed for table cancer_mama_1.procedimentos_e_pacientes
error during vacuum for table cancer_mama_1.cancer_ordered
error during vacuum for table cancer_mama_1.dados_municipios_mensal
error during vacuum for table cancer_mama_1.diagnosticos_por_estadiamento_municipio
